# 03 — Populate a sample A-box (Subjects, Visits, Observations)

This notebook builds an **A-box** from a longitudinal table (PPMI-like), using:

- the bridge ontology: `ontologies/ppmi_bridge.ttl`
- the mapping file: `mapping/ppmi_pdon_pmdo_mapping.csv`

It creates:
- `ppmi:Subject` individuals (by `PATNO`)
- `ppmi:Visit` individuals (by `PATNO` + `EVENT_ID`)
- `ppmi:Observation` individuals for mapped variables
- `ppmi:DiagnosisObservation` for **PRIMDIAG** codes (baseline-only, by default)

**Input data**: you may provide the **full PPMI file** (many columns). The script will read it and only use columns required by the mapping + meta fields.

**Output**: `output/ppmi_abox_sample.ttl`


In [23]:
!pip -q install rdflib pandas owlrl

## 1) Project root (Drive recommended)

In [24]:
from pathlib import Path
USE_DRIVE = True
if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')
  PROJECT_DIR = Path('/content/drive/MyDrive/ppmi-ontology-alignment')
else:
  PROJECT_DIR = Path('/content/ppmi-ontology-alignment')

ONT_DIR = PROJECT_DIR / 'ontologies'
MAP_DIR = PROJECT_DIR / 'mapping'
DATA_DIR = PROJECT_DIR / 'data'
OUT_DIR = PROJECT_DIR / 'output'
for p in [ONT_DIR, MAP_DIR, DATA_DIR, OUT_DIR]:
  p.mkdir(parents=True, exist_ok=True)

BRIDGE_PATH = ONT_DIR / 'ppmi_bridge.ttl'
MAP_PATH    = MAP_DIR / 'ppmi_pdon_pmdo_mapping.csv'
DEMO_PATH   = DATA_DIR / 'ppmi_demo.tsv'
ABOX_OUT    = OUT_DIR / 'ppmi_abox_sample.ttl'

print('PROJECT_DIR:', PROJECT_DIR)
print('Bridge exists :', BRIDGE_PATH.exists(), BRIDGE_PATH)
print('Mapping exists:', MAP_PATH.exists(), MAP_PATH)
print('Demo exists   :', DEMO_PATH.exists(), DEMO_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_DIR: /content/drive/MyDrive/ppmi-ontology-alignment
Bridge exists : True /content/drive/MyDrive/ppmi-ontology-alignment/ontologies/ppmi_bridge.ttl
Mapping exists: True /content/drive/MyDrive/ppmi-ontology-alignment/mapping/ppmi_pdon_pmdo_mapping.csv
Demo exists   : True /content/drive/MyDrive/ppmi-ontology-alignment/data/ppmi_demo.tsv


## 2) Load mapping CSV

In [25]:
import pandas as pd
from google.colab import files
import shutil

if not MAP_PATH.exists():
  print('Upload ppmi_pdon_pmdo_mapping.csv')
  up = files.upload()
  for fn in up.keys():
    shutil.move(f'/content/{fn}', str(MAP_PATH if fn=='ppmi_pdon_pmdo_mapping.csv' else (MAP_DIR/fn)))

mapping = pd.read_csv(MAP_PATH)

# Normalise key fields
for col in ['Category','Variable','Code','Decode','MappingBucket','TargetIRI','How','Confidence']:
  if col in mapping.columns:
    mapping[col] = mapping[col].astype(str)

mapping['Variable'] = mapping['Variable'].astype(str).str.strip()
mapping['TargetIRI'] = mapping['TargetIRI'].astype(str).str.strip()
mapping.head(10)


,Category,Variable,Code,Decode,MappingBucket,TargetIRI,How,Confidence
0,SubjectMeta,PATNO,nan,nan,SUBJECT_META,nan,Subject identifier; used to mint ppmi:Subject ...,struct
1,SubjectMeta,COHORT,nan,nan,SUBJECT_META,nan,Subject cohort label; attach as data property ...,struct
2,SubjectMeta,subgroup,nan,nan,SUBJECT_META,nan,Subject subgroup label; attach as data propert...,struct
3,SubjectMeta,SEX,nan,nan,SUBJECT_META,nan,Subject sex; attach as data property on Subject.,struct
4,SubjectMeta,EDUCYRS,nan,nan,SUBJECT_META,nan,Years of education; attach as data property on...,struct
5,SubjectMeta,fampd_bin,nan,nan,SUBJECT_META,nan,Family history of PD (binary); attach as data ...,struct
6,Genetics,APOE,nan,nan,OBS_STRUCT,nan,Genetic observation (categorical); store as Ob...,struct
7,VisitMeta,EVENT_ID,nan,nan,VISIT_META,nan,Visit identifier; used to mint ppmi:Visit IRI ...,struct
8,VisitMeta,visit_date,nan,nan,VISIT_META,nan,Visit timestamp; attach to Visit (xsd:gYearMon...,struct
9,VisitMeta,YEAR,nan,nan,VISIT_META,nan,Visit time index; attach to Visit (integer).,struct


## 3) Provide / upload longitudinal data

Expected format: **TSV** with one row per (PATNO, EVENT_ID).

Save as `data/ppmi_demo.tsv` (tab-separated).

You can also use a larger/full file; the script will only use needed columns.


In [26]:
from google.colab import files
import shutil

if not DEMO_PATH.exists():
  print('Upload ppmi_demo.tsv (tab-separated)')
  up = files.upload()
  for fn in up.keys():
    dest = DEMO_PATH if fn=='ppmi_demo.tsv' else (DATA_DIR/fn)
    shutil.move(f'/content/{fn}', str(dest))
  print('Saved to:', DEMO_PATH)

# NOTE: we will load the TSV after computing required columns (usecols) in Step 4.
print('TSV path ready:', DEMO_PATH)


TSV path ready: /content/drive/MyDrive/ppmi-ontology-alignment/data/ppmi_demo.tsv


## 4) Build A-box (RDF)

In [27]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD
import re

PPMI = Namespace('http://example.org/ppmi-ontology-alignment#')

# -----------------------------
# Helpers
# -----------------------------

def mint_subject(patno: str) -> URIRef:
  return PPMI[f'subject/{patno}']

def mint_visit(patno: str, event_id: str) -> URIRef:
  return PPMI[f'visit/{patno}/{event_id}']

def mint_obs(patno: str, event_id: str, var: str) -> URIRef:
  safe = re.sub(r'[^A-Za-z0-9_\-\.]+','_', var)
  return PPMI[f'obs/{patno}/{event_id}/{safe}']

def parse_gYearMonth(s: str):
  if s is None:
    return None
  m = re.match(r'^(\d{1,2})/(\d{4})$', str(s).strip())
  if not m:
    return None
  mm, yy = int(m.group(1)), int(m.group(2))
  if mm < 1 or mm > 12:
    return None
  return f'{yy:04d}-{mm:02d}'

def _clean(v):
  if v is None:
    return ''
  s = str(v).strip()
  return '' if s.lower()=='nan' else s

def literal_best(x):
  import pandas as pd
  if x is None:
    return None
  if isinstance(x, float) and pd.isna(x):
    return None
  s = str(x).strip()
  if s == '' or s.lower() == 'nan':
    return None

  # Integers
  if re.fullmatch(r'[-+]?\d+', s):
    try:
      return Literal(int(s), datatype=XSD.integer)
    except Exception:
      pass

  # Decimals / scientific
  if re.fullmatch(r'[-+]?\d*\.\d+(?:[eE][-+]?\d+)?', s) or re.fullmatch(r'[-+]?\d+\.\d*', s):
    try:
      return Literal(float(s), datatype=XSD.decimal)
    except Exception:
      pass

  return Literal(s, datatype=XSD.string)

def normalize_code_for_lookup(x):
  """Normalise categorical code values to a stable string key.
  Handles inputs like '1', '1.0', 1, 1.0 -> '1'.
  Leaves non-numeric strings untouched.
  """
  if x is None:
    return ''
  s = str(x).strip()
  if s == '' or s.lower() == 'nan':
    return ''
  # numeric?
  if re.fullmatch(r'[-+]?\d+(?:\.\d+)?', s):
    try:
      f = float(s)
      if abs(f - round(f)) < 1e-9:
        return str(int(round(f)))
      # keep compact form (no trailing zeros)
      return ('%s' % f).rstrip('0').rstrip('.')
    except Exception:
      return s
  return s

# -----------------------------
# Mapping lookups
# -----------------------------

map_rows = mapping.copy()

# variable -> (target IRI) for single-target variables
var2target = {}
for _, r in map_rows.iterrows():
  var = _clean(r.get('Variable'))
  tgt = _clean(r.get('TargetIRI'))
  if var and tgt:
    # If multiple rows exist for same Variable (e.g., PRIMDIAG), we do NOT overwrite here.
    if var != 'PRIMDIAG':
      var2target[var] = tgt

# PRIMDIAG: code -> (decode, targetIRI)
prim = map_rows[map_rows['Variable'].astype(str).str.strip() == 'PRIMDIAG'].copy()
prim_lookup = {}
for _, r in prim.iterrows():
  code_key = normalize_code_for_lookup(r.get('Code'))
  if not code_key:
    continue
  dec = _clean(r.get('Decode'))
  tgt = _clean(r.get('TargetIRI'))
  prim_lookup[code_key] = (dec, tgt)

# Identify DATSCAN vars and "observation" vars
# Note: we include subject/visit meta columns in the data file but we must not create Observations for them.

datscan_vars = set(map_rows.loc[map_rows['Category'].astype(str).str.upper().eq('DATSCAN'), 'Variable'].astype(str).str.strip())

# Variables that should become Observations in the A-box
obs_vars = sorted(set(
  map_rows.loc[
    map_rows['MappingBucket'].astype(str).str.upper().isin(['PMDO','MDO','PDON','PPMI/PMDO','OBS_STRUCT']),
    'Variable'
  ].astype(str).str.strip()
))

# Exclude structural/meta buckets from observations (best effort from CSV)
meta_buckets = set(['SUBJECT_META','VISIT_META'])
meta_vars = set(map_rows.loc[map_rows['MappingBucket'].astype(str).str.upper().isin(meta_buckets), 'Variable'].astype(str).str.strip())
obs_vars = [v for v in obs_vars if v and v not in meta_vars]

# --- HARD EXCLUSION: never create Observations for these known meta fields ---
SUBJECT_META = ['COHORT','subgroup','SEX','EDUCYRS','fampd_bin','APOE']
VISIT_META   = ['EVENT_ID','visit_date','YEAR','age_at_visit']
obs_vars = [v for v in obs_vars if v not in set(SUBJECT_META + VISIT_META + ['PATNO'])]

# -----------------------------
# Load longitudinal TSV (usecols) - supports full PPMI TSV safely
# -----------------------------

required_cols = set(['PATNO','EVENT_ID','PRIMDIAG']) | set(SUBJECT_META) | set(VISIT_META) | set(obs_vars)

header_cols = pd.read_csv(DEMO_PATH, sep='\t', nrows=0).columns
usecols = [c for c in header_cols if c in required_cols]
missing = sorted(required_cols - set(usecols))

print('Using cols:', len(usecols))
if missing:
  print('Missing cols (ok if not in this file):', missing[:30], '...' if len(missing) > 30 else '')

# Read as strings first to avoid float coercions on categorical codes
# We'll cast values later with literal_best.
demo = pd.read_csv(DEMO_PATH, sep='\t', dtype=str, usecols=usecols)
print('Rows:', len(demo), 'Cols:', len(demo.columns))

# -----------------------------
# Graph + T-box terms
# -----------------------------

g = Graph()
g.bind('ppmi', PPMI)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('owl', OWL)
g.bind('xsd', XSD)

C_Subject = PPMI['Subject']
C_Visit = PPMI['Visit']
C_Obs = PPMI['Observation']
C_DxObs = PPMI['DiagnosisObservation']
C_ImgObs = PPMI['ImagingObservation']

P_hasVisit = PPMI['hasVisit']
P_hasObs = PPMI['hasObservation']
P_observes = PPMI['observesConcept']
P_refersPdon = PPMI['refersToPdonConcept']
P_relRegion = PPMI['relatesToRegion']

P_hasValue = PPMI['hasValue']
P_hasCode  = PPMI['hasCode']
P_hasDecode= PPMI['hasDecode']
P_visitDate= PPMI['visitDate']
P_visitYear= PPMI['visitYear']
P_ageAtVisit=PPMI['ageAtVisit']


def add_subject_meta(subj, row):
  for v in SUBJECT_META:
    if v in row.index:
      lit = literal_best(row[v])
      if lit is not None:
        g.add((subj, PPMI[f'subjectMeta/{v}'], lit))

def add_visit_meta(visit, row):
  # visit_date: prefer gYearMonth if in MM/YYYY
  if 'visit_date' in row.index:
    s = str(row['visit_date']).strip() if _clean(row['visit_date']) else ''
    gy = parse_gYearMonth(s)
    if gy:
      g.add((visit, P_visitDate, Literal(gy, datatype=XSD.gYearMonth)))
    elif s:
      g.add((visit, P_visitDate, Literal(s, datatype=XSD.string)))

  if 'YEAR' in row.index:
    lit = literal_best(row['YEAR'])
    if lit is not None:
      g.add((visit, P_visitYear, lit))

  if 'age_at_visit' in row.index:
    lit = literal_best(row['age_at_visit'])
    if lit is not None:
      g.add((visit, P_ageAtVisit, lit))

# -----------------------------
# Choose scope: one subject sample or full dataset
# -----------------------------

# By default, create a small A-box for one PATNO (first row) to keep output manageable.
# If you want the full dataset, set BUILD_ALL = True.
BUILD_ALL = False

# Optional safety cap when BUILD_ALL=True (set to None to disable)
MAX_SUBJECTS = None

if 'PATNO' not in demo.columns or 'EVENT_ID' not in demo.columns:
  raise ValueError('Input TSV must contain at least PATNO and EVENT_ID columns')

if BUILD_ALL:
  work_df = demo.copy()
  if MAX_SUBJECTS:
    pats = work_df['PATNO'].dropna().astype(str).str.strip().unique().tolist()
    keep = set(pats[:MAX_SUBJECTS])
    work_df = work_df[work_df['PATNO'].astype(str).str.strip().isin(keep)].copy()
    print('BUILD_ALL=True with MAX_SUBJECTS ->', MAX_SUBJECTS, 'subjects, rows:', len(work_df))
  else:
    print('BUILD_ALL=True -> using all rows:', len(work_df))
else:
  SAMPLE_PATNO = str(demo['PATNO'].iloc[0]).strip()
  work_df = demo[demo['PATNO'].astype(str).str.strip() == SAMPLE_PATNO].copy()
  print('BUILD_ALL=False -> Sample PATNO:', SAMPLE_PATNO, 'rows:', len(work_df))

# Option: PRIMDIAG baseline-only
PRIMDIAG_BASELINE_ONLY = True

# -----------------------------
# Populate
# -----------------------------

for _, row in work_df.iterrows():
  patno = str(row['PATNO']).strip()
  event = str(row['EVENT_ID']).strip()
  if patno == '' or event == '':
    continue

  subj = mint_subject(patno)
  visit = mint_visit(patno, event)

  g.add((subj, RDF.type, C_Subject))
  g.add((visit, RDF.type, C_Visit))
  g.add((subj, P_hasVisit, visit))

  add_subject_meta(subj, row)
  add_visit_meta(visit, row)

  # Create observations for mapped variables
  for var in obs_vars:
    if var not in row.index:
      continue

    # Guard: meta fields must never become Observations
    if var in SUBJECT_META or var in VISIT_META or var in ('PATNO', 'EVENT_ID'):
      continue

    # Handle PRIMDIAG separately
    if var == 'PRIMDIAG':
      if PRIMDIAG_BASELINE_ONLY and event != 'BL':
        continue

      raw = row.get('PRIMDIAG', None)
      code_key = normalize_code_for_lookup(raw)
      if not code_key:
        continue

      obs = mint_obs(patno, event, var)
      g.add((visit, P_hasObs, obs))

      g.add((obs, RDF.type, C_DxObs))
      g.add((obs, P_hasCode, Literal(code_key, datatype=XSD.string)))

      dec, tgt = prim_lookup.get(code_key, ('',''))
      if dec:
        g.add((obs, P_hasDecode, Literal(dec, datatype=XSD.string)))
      if tgt:
        g.add((obs, P_refersPdon, URIRef(tgt)))

      continue

    # Normal observation variables
    val_lit = literal_best(row[var])
    if val_lit is None:
      continue

    obs = mint_obs(patno, event, var)
    g.add((visit, P_hasObs, obs))

    # DATSCAN region variables (MIA_*) -> ImagingObservation + relatesToRegion
    if var in datscan_vars and var.startswith('MIA_'):
      g.add((obs, RDF.type, C_ImgObs))
      tgt_region = var2target.get(var)
      if tgt_region:
        g.add((obs, P_relRegion, URIRef(tgt_region)))
    else:
      g.add((obs, RDF.type, C_Obs))

    g.add((obs, P_hasValue, val_lit))

    tgt = var2target.get(var)
    if tgt:
      g.add((obs, P_observes, URIRef(tgt)))

print('A-box triples:', len(g))
print('PRIMDIAG codes in mapping lookup:', len(prim_lookup))


Using cols: 33
Missing cols (ok if not in this file): ['mia_lowput_expected'] 
Rows: 6 Cols: 33
BUILD_ALL=False -> Sample PATNO: 100001 rows: 6
A-box triples: 382
PRIMDIAG codes in mapping lookup: 11


## 5) Serialise A-box to Turtle

In [28]:
g.serialize(destination=str(ABOX_OUT), format='turtle')
print('Wrote:', ABOX_OUT)


Wrote: /content/drive/MyDrive/ppmi-ontology-alignment/output/ppmi_abox_sample.ttl


## 6) Quick checks (SPARQL)

- A richer sample listing (value/code/decode/concepts)
- Sanity: PRIMDIAG mapped vs unmapped
- Sanity: PRIMDIAG baseline-only violations
- Sanity: APOE should NOT appear as Observation


In [30]:
q = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT ?obs ?type ?value ?code ?decode ?concept ?pdon
WHERE {
  ?visit ppmi:hasObservation ?obs .
  OPTIONAL { ?obs a ?type . }
  OPTIONAL { ?obs ppmi:hasValue ?value }
  OPTIONAL { ?obs ppmi:hasCode ?code }
  OPTIONAL { ?obs ppmi:hasDecode ?decode }
  OPTIONAL { ?obs ppmi:observesConcept ?concept }
  OPTIONAL { ?obs ppmi:refersToPdonConcept ?pdon }
}
LIMIT 50
'''
for row in g.query(q):
  print(row)

print('\n--- Counts ---')
q_counts = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT
  (COUNT(DISTINCT ?s) AS ?nSubjects)
  (COUNT(DISTINCT ?v) AS ?nVisits)
  (COUNT(DISTINCT ?o) AS ?nObs)
  (COUNT(DISTINCT ?dx) AS ?nDxObs)
WHERE {
  OPTIONAL { ?s a ppmi:Subject . }
  OPTIONAL { ?v a ppmi:Visit . }
  OPTIONAL { ?o a ppmi:Observation . }
  OPTIONAL { ?dx a ppmi:DiagnosisObservation . }
}
'''
print(list(g.query(q_counts))[0])

print('\n--- PRIMDIAG totals ---')
q_dx_total = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT (COUNT(DISTINCT ?dx) AS ?nDxTotal)
WHERE {
  ?dx a ppmi:DiagnosisObservation ;
      ppmi:hasCode ?code .
}
'''
print(list(g.query(q_dx_total))[0])

print('\n--- PRIMDIAG with PDON link ---')
q_dx_with = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT (COUNT(DISTINCT ?dx) AS ?nDxWithPdon)
WHERE {
  ?dx a ppmi:DiagnosisObservation ;
      ppmi:hasCode ?code ;
      ppmi:refersToPdonConcept ?pdon .
}
'''
print(list(g.query(q_dx_with))[0])

print('\n--- PRIMDIAG without PDON link ---')
q_dx_without = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT (COUNT(DISTINCT ?dx) AS ?nDxNoPdon)
WHERE {
  ?dx a ppmi:DiagnosisObservation ;
      ppmi:hasCode ?code .
  FILTER NOT EXISTS { ?dx ppmi:refersToPdonConcept ?pdon . }
}
'''
print(list(g.query(q_dx_without))[0])

print('\n--- PRIMDIAG baseline-only violations (should be 0 rows) ---')
q_dx_not_bl = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT ?visit ?dx
WHERE {
  ?visit ppmi:hasObservation ?dx .
  ?dx a ppmi:DiagnosisObservation .
  FILTER(!CONTAINS(STR(?visit), "/BL"))
}
LIMIT 20
'''
rows = list(g.query(q_dx_not_bl))
print('Dx not BL:', len(rows))
for r in rows:
  print(r)

print('\n--- APOE must NOT be an Observation (should be 0 rows) ---')
q_apoe_obs = '''
PREFIX ppmi: <http://example.org/ppmi-ontology-alignment#>
SELECT ?visit ?obs
WHERE {
  ?visit ppmi:hasObservation ?obs .
  FILTER(CONTAINS(STR(?obs), "/APOE"))
}
LIMIT 20
'''
rows = list(g.query(q_apoe_obs))
print('APOE as Observation:', len(rows))
for r in rows:
  print(r)


(rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#obs/100001/BL/MCI_testscores'), rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#Observation'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), None, None, None, None)
(rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#obs/100001/BL/MIA_CAUDATE_BILAT'), rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#ImagingObservation'), rdflib.term.Literal('0.7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')), None, None, rdflib.term.URIRef('http://www.case.edu/MDO#Caudate'), None)
(rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#obs/100001/BL/MIA_CAUDATE_L'), rdflib.term.URIRef('http://example.org/ppmi-ontology-alignment#ImagingObservation'), rdflib.term.Literal('0.9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')), None, None, rdflib.term.URIRef('http://www.case.edu/MDO#Caudate'